In [380]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('precision', 3)

In [381]:
data = pd.read_csv('raw/jj-flowcharts-2011-2015.csv')

In [382]:
data.replace('--', np.nan, inplace=True)

In [383]:
yrs = ['2011', '2012', '2013', '2014', '2015']

for yr in yrs:
    try:
        data[yr] = pd.to_numeric(data[yr])
    except TypeError:
        print('not an object')
        continue

In [384]:
data['pct_chg'] = ((data['2015'] - data['2011']) / data['2011']) * 100

The flowcharts don't break down which guilty verdicts are a result of trial and which are pleas. But we can infer this delineation by subtracting not guilty verdicts from total trials to get the number of guilty verdicts resulting from trials, and by extension, the number resulting from pleas.

![title](fc.png)

In 2013 and 2015 (shown), the number of pleas is greater than guilty verdicts. The difference between guilty verdicts as a result of pleas and total pleas is logged as a column with an action, '???'.

In [385]:
trial = data.loc[data['action'] == 'trial']
not_guilty = data.loc[data['action'] == 'not guilty']
og_guilty = data.loc[(data['action'] == 'guilty') & (data['parent'].str.contains('plea'))]

trial_guilty = {'action': 'guilty', 'parent': 'trial'}
plea_guilty = {'action': 'guilty', 'parent': 'plea'}
mystery = {'action': '???', 'parent': 'plea'}

for yr in yrs:
    try:
        trial_guilty[yr] = int(trial[yr]) - int(not_guilty[yr])
        plea_guilty[yr] = int(og_guilty[yr]) - trial_guilty[yr]
        mystery[yr] = int(data.loc[data['action'] == 'plea'][yr]) - plea_guilty[yr]
    except ValueError:
        trial_guilty[yr] = np.nan
        plea_guilty[yr] = np.nan
        mystery[yr] = np.nan

trial_guilty['pct_chg'] = ((trial_guilty['2015'] - trial_guilty['2011']) / trial_guilty['2011']) * 100
plea_guilty['pct_chg'] = ((plea_guilty['2015'] - plea_guilty['2011']) / plea_guilty['2011']) * 100
mystery['pct_chg'] = np.nan


for i in (trial_guilty, plea_guilty, mystery):
    data = data.append(i, ignore_index=True)

In [392]:
data = pd.concat([data[:26], data[33:], data[26:33]]).reset_index(drop=True).drop(29)

Arrests have decreased by 25 percent since 2011, which means looking at raw numbers isn't necessarily going to be best for comparison. So let's add columns for each action as a percentage of its parent and of total arrests.

In [393]:
d = []

arrests_11 = data.iloc[0]['2011']
arrests_15 = data.iloc[0]['2015']

for ind, row in data.iterrows():
    action = row['action']
    parent = row['parent']
    value_11 = row['2011']
    value_15 = row['2015']
    if action != '???':
        try:
            parent_value_11 = int(data.loc[data['action'] == parent]['2011'])
            parent_value_15 = int(data.loc[data['action'] == parent]['2015'])
        except TypeError: # a is nan, or weird parent (my bad)
            continue
        d.append((action, parent, value_11, value_15, row['pct_chg'], 
                 (value_11 / parent_value_11 * 100), (value_15 / parent_value_15 * 100), 
                 (value_11 / arrests_11 * 100), (value_15 / arrests_15 * 100)))

c = pd.DataFrame(d, columns=['action', 'parent', '2011', '2015', 'pct_chg', 
                             'share_act_2011', 'share_act_2015',
                             'share_arrests_2011', 'share_arrests_2015'])

c = c.drop(5)

Outcomes occuring to a greater proportion of juvenile offenders who reach the parent decision point:
  * Officers recommended detention for 46 percent of juvenile offenders whom they chose to prosecute in 2015, up from 33 percent in 2011.
  * Judges did not detain 25 percent of juvenile offenders whom they screened in 2015, up from 20 percent in 2011.
  * Judges placed 24 percent of juvenile offenders whom they screened in shelters in 2015, up from 2 percent in 2011.
  * ?? detained 22 percent of juvenile offenders for whom they found probable cause in 2015, up from 11 percent in 2011.
  * 53 percent of juvenile offenders were found not guilty at trial in 2015, up from 33 percent in 2011.

In [394]:
c.loc[c['share_act_2015'] - c['share_act_2011'] >=5]

,action,parent,2011,2015,pct_chg,share_act_2011,share_act_2015,share_arrests_2011,share_arrests_2015
0,diversion by police,arrest,7932,6967.0,-12.166,30.554,36.140,30.554,36.140
3,recommend detention,prosecute,5917,5888.0,-0.490,33.010,46.141,22.792,30.543
7,don't detain,detention screening,1214,1528.0,25.865,20.517,25.951,4.676,7.926
8,place in shelter,detention screening,113,1425.0,1161.062,1.910,24.202,0.435,7.392
17,detain,probable cause found,725,1080.0,48.966,11.295,22.519,2.793,5.602
22,not guilty,trial,100,385.0,285.000,33.557,53.547,0.385,1.997


Outcomes occurring to a greater proportion of juvenile offenders who enter the criminal justice system:
  * 36 percent of juveniles arrested were diverted via station adjustment in 2015, up from 30 percent in 2011
  * Officers recommended detention for 30 percent of juveniles arrested, up from 22 percent in 2011
  * 7 percent of juveniles arrested were placed in shelters after intitial detention screening in 2015, up from less than 1 percent in 2011
  * 17 percent of juveniles arrested eventually pled guilty in 2015, up from 12 percent in 2011

In [395]:
c.loc[c['share_arrests_2015'] - c['share_arrests_2011'] >= 5]

,action,parent,2011,2015,pct_chg,share_act_2011,share_act_2015,share_arrests_2011,share_arrests_2015
0,diversion by police,arrest,7932,6967.0,-12.166,30.554,36.140,30.554,36.140
3,recommend detention,prosecute,5917,5888.0,-0.490,33.010,46.141,22.792,30.543
8,place in shelter,detention screening,113,1425.0,1161.062,1.910,24.202,0.435,7.392
21,guilty,plea,3242,3400.0,4.874,100.000,83.579,12.488,17.637


Actions occuring more frequently relative to themselves:

In [396]:
c.loc[c['pct_chg'] > 0]

,action,parent,2011,2015,pct_chg,share_act_2011,share_act_2015,share_arrests_2011,share_arrests_2015
7,don't detain,detention screening,1214,1528.0,25.865,20.517,25.951,4.676,7.926
8,place in shelter,detention screening,113,1425.0,1161.062,1.910,24.202,0.435,7.392
16,probable cause not found,probable cause hearing,44,109.0,147.727,0.681,2.222,0.169,0.565
17,detain,probable cause found,725,1080.0,48.966,11.295,22.519,2.793,5.602
20,guilty,trial,198,334.0,68.687,66.443,46.453,0.763,1.733
21,guilty,plea,3242,3400.0,4.874,100.000,83.579,12.488,17.637
22,not guilty,trial,100,385.0,285.000,33.557,53.547,0.385,1.997


In [397]:
c

,action,parent,2011,2015,pct_chg,share_act_2011,share_act_2015,share_arrests_2011,share_arrests_2015
0,diversion by police,arrest,7932,6967.0,-12.166,30.554,36.140,30.554,36.140
1,prosecute,arrest,17925,12761.0,-28.809,69.046,66.195,69.046,66.195
2,transfer to adult court,arrest,104,NaN,NaN,0.401,NaN,0.401,NaN
3,recommend detention,prosecute,5917,5888.0,-0.490,33.010,46.141,22.792,30.543
4,don't detain,prosecute,12008,6873.0,-42.763,66.990,53.859,46.254,35.652
6,detain,detention screening,3401,2935.0,-13.702,57.478,49.847,13.100,15.225
7,don't detain,detention screening,1214,1528.0,25.865,20.517,25.951,4.676,7.926
8,place in shelter,detention screening,113,1425.0,1161.062,1.910,24.202,0.435,7.392
9,24-hour non-secure custody,detention screening,1189,NaN,NaN,20.095,NaN,4.580,NaN
10,transfer to adult court,case screening,0,0.0,NaN,0.000,0.000,0.000,0.000


In [398]:
data.to_csv('processed/jj-flowcharts-2011-2015.csv', index=False)